## **Import library**

In [2]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt

## **Define function for normalization and image display**

In [3]:
#image normalization
def normalization(img, range):
  normed_img = img/(img.max()/range)
  return normed_img

In [4]:
def plotImage(image, title):
  plt.imshow(image, 'gray', vmin = 0, vmax = 255)
  plt.title(title)
  fig = plt.gcf()
  fig.set_size_inches(13,13)
  plt.show()

In [5]:
def convolve2d(image, kernel, stride = 1):
  kernel = np.flipud(np.fliplr(kernel))

  k_sizeX, k_sizeY = kernel.shape

  im_sizeX, im_sizeY = image.shape

  padding = int(np.floor((k_sizeX-1)/2)) # padding = ((k-1) / 2)

  #output image (convolved with image)
  new_image = np.zeros((im_sizeX + 2*padding, im_sizeY + 2*padding))
  new_image[padding: im_sizeX+padding, padding: im_sizeY + padding] = image[:,:]

  output = np.zeros(new_image.shape)

  new_im_sizeX, new_im_sizeY = new_image.shape
  for y in range(new_im_sizeY):
    if y > new_im_sizeY-k_sizeY:
      break

    for x in range(new_im_sizeX):
      if x > new_im_sizeX-k_sizeX:
        break
      
      if( y % stride == 0 and x%stride == 0):
        
        output[int(np.floor((2*x+k_sizeX)/2)),int(np.floor((2*y+k_sizeY)/2))] = (kernel * new_image[x:x+k_sizeX, y:y+k_sizeY]).sum()

  return output

In [6]:
def grey_scale(img):
    R, G, B = img[:,:,0], img[:,:,1], img[:,:,2]
    imgGray = 0.2989 * R + 0.5870 * G + 0.1140 * B
    return imgGray

## **Define Prewitt Operator**

In [7]:
Px = np.array([[1, 0, -1],
               [1, 0, -1],
               [1, 0, -1]])

Py = np.array([[1, 1, 1],
               [0, 0, 0],
               [-1, -1, -1]])

## **HOG Feature **

In [8]:
def get_bin(cell_theta, cell_M):
    #calculate the histogram of the each cell
    bin_size = 9
    bin_degree = 180/bin_size
    hist = np.zeros(9)
    for i in range(cell_theta.shape[0]):
        for j in range(cell_theta.shape[1]):
            bin_index = int((cell_theta[i, j] + 10) // 20)
     
            v_1 = cell_M[i, j] * (bin_degree * (bin_index + 1) - 10 - cell_theta[i,j])/bin_degree
            v_2 = cell_M[i, j] * (cell_theta[i,j] - bin_degree * bin_index + 10)/bin_degree
            
            hist[bin_index] += v_1
            if bin_index +1 <= 8:
                hist[bin_index+1] += v_2
    return hist

def block_normalization(block_vec):
    temp = np.sqrt(np.sum(np.power((block_vec),2)))
    if temp == 0:
        return block_vec
    return block_vec/temp


def get_hist_cell(theta, M):
    #get the histogram of the each cell
    width, height = M.shape[0], M.shape[1]
    bin_size = 9
    cell_size = 8
    step = 8
    hist_vector = np.zeros((int(width/cell_size), int(height/cell_size), bin_size))
    for i in range(hist_vector.shape[0]):
        for j in range(hist_vector.shape[1]):
            cell_magnitude = M[i * cell_size: (i+1) * cell_size, j*cell_size : (j+1) *cell_size]
            cell_theta = theta[i * cell_size: (i+1) * cell_size, j*cell_size : (j+1) *cell_size]
            cell_hist = get_bin(cell_theta, cell_magnitude)
            hist_vector[i][j] = cell_hist
            


# get the histogram of the whole image
    hog_vector = []
    for i in range(int(width/cell_size) - 1):
        for j in range(int(height/cell_size) -1):
            block_vec = []
            block_vec.extend(hist_vector[i][j])
            block_vec.extend(hist_vector[i][j+1])
            block_vec.extend(hist_vector[i+1][j])
            block_vec.extend(hist_vector[i+1][j+1])
            hog_vector.extend(block_normalization(block_vec))
    return hog_vector

## **Feed into the Neuro Network**

In [29]:
class nn():
    def __init__(self, X, layer_size, test_X):
        np.random.seed(1)
        self.X = X.T
        self.test_X = test_X.T
        # self.y = y
        self.ground_truth = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0]])
        self.num = 0
        self.test_ground_truth = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])
        self.y_pred = np.zeros((self.ground_truth.shape[1],1))
        self.sam = self.ground_truth.shape[1]
        self.learning_rate = 0.00003
        # self.layer_num = 2
        self.layer_size = layer_size
        self.weights_1 = 2 * np.random.rand(self.layer_size, X.shape[1]) - 1
        self.weights_2 = 2 * np.random.rand(1, self.layer_size) - 1
        self.bias_1 = np.random.rand(self.layer_size,1)
        self.bias_2 = np.random.rand(1,1)
        self.test_loss = 0

        self.loss = 0
        # self.layer_output = feed_forward(X, weights)

    def dSigmoid(self,x):
        s = 1/(1+np.exp(-x))
        return np.multiply(s,(1-s))

    def dRelu(self,x):
        x[ x <= 0] = 0
        x[x > 0] = 1
        return x
    
    def sigmoid_function(self,x):
        return 1/(1+np.exp(-x))

    def relu_function(self,x):
        return np.maximum(0,x)

    def loss_function(self, y_true, y_pred):
        y_pred[y_pred == 1] = 1 - (1e-8)
        loss = (1./self.sam) * (-np.dot(y_true,np.log(y_pred).T) - np.dot(1-y_true, np.log(1-y_pred).T))    
        return loss

    def train(self):
        test_brek = False
        for iteration in range(10000):
            #feed forward
            if iteration % 10 == 0:
                test_loss = self.loss_function(self.test_ground_truth, self.pred(self.test_X))
                # if the loss for test increase, we stop the iteration
                # print(self.y_pred)
                if test_loss > self.test_loss:
                    self.num += 1
                else:
                    self.num = 0
                if self.num == 3:
                    break
                self.test_loss = test_loss
                
                print("iteration" + str(iteration) + ":" + str(self.y_pred) + "Loss" + str(self.loss) + "test loss:" + str(self.test_loss))
            before_1 = np.dot(self.weights_1, self.X) + self.bias_1
            layer1_output = self.relu_function(before_1)
            before_2 = np.dot(self.weights_2, layer1_output) + self.bias_2
            layer2_output = self.sigmoid_function(before_2)
            layer2_output[layer2_output == 1] = 1 - (1e-8)
            self.y_pred = layer2_output
            self.loss = self.loss_function(self.ground_truth, self.y_pred)


            d_y_pred = - np.divide(self.ground_truth, self.y_pred) + np.divide(1-self.ground_truth, 1-self.y_pred)
            d_before_2 = np.multiply(d_y_pred,self.dSigmoid(before_2))
            d_layer1_output = np.dot(self.weights_2.T, d_before_2)
            d_weight_2 = 1./layer1_output.shape[1] * np.dot(d_before_2, layer1_output.T)
            d_bais_2 = 1./layer1_output.shape[1] * np.dot(d_before_2, np.ones([d_before_2.shape[1], 1]))
            d_before_1 = np.multiply(d_layer1_output,self.dRelu(before_1))
            d_input = np.dot(self.weights_1.T, d_before_1)
            d_weight_1 = 1./self.X.shape[1] * np.dot(d_before_1, self.X.T)
            d_bais_1 = 1./self.X.shape[1] * np.dot(d_before_1, np.ones([d_before_1.shape[1],1]))
            self.weights_1 = self.weights_1 - self.learning_rate * d_weight_1
            self.bias_1 = self.bias_1 - self.learning_rate * d_bais_1
            self.weights_2 = self.weights_2 - self.learning_rate * d_weight_2
            self.bias_2 = self.bias_2 - self.learning_rate * d_bais_2
            
            

    def pred(self, input):
            before_1 = np.dot(self.weights_1, input) + self.bias_1
            layer1_output = self.relu_function(before_1)
            before_2 = np.dot(self.weights_2, layer1_output) + self.bias_2
            layer2_output = self.sigmoid_function(before_2)
            return layer2_output

## **Prepare the training image**

In [11]:
def prepare_image(img_name):
    img = cv2.imread(img_name) 
    img = grey_scale(img)
    Gx = convolve2d(img,Px)
    Gy = convolve2d(img,Py)
    M = np.sqrt(Gx*Gx + Gy*Gy)
    M = np.round(normalization(M, 255))
    theta = np.zeros(Gx.shape)
    for i in range(theta.shape[0]):
        for j in range(theta.shape[1]):
            if Gy[i,j] == 0 and Gx[i,j] == 0:
                theta[i,j] = 0
            elif Gx[i,j] == 0:
                theta[i,j] = 90
            else:
                theta[i,j] = np.arctan2(Gy[i,j], Gx[i,j]) * 180 / np.pi
                if theta[i,j] < -10:
                    theta[i,j] += 180
                elif theta [i,j] >= 170:
                    theta[i,j] -= 180
    for i in range(theta.shape[0]):
        for j in range(theta.shape[1]):
            if str(theta[i,j]) == "nan":
                print(true)
    hog_vector= get_hist_cell(theta, M)
    return hog_vector

## **Read in the training data, First positive, then negative**

In [12]:
# from skimage import io 
import os
data_input = [] 
path = "/Users/shenmengjie/Desktop/Computer Vision/project2/data/Train_Positive/"
path_2 = "/Users/shenmengjie/Desktop/Computer Vision/project2/data/Train_Negative/"

file_dir = os.listdir(path)
for file in file_dir:
    if not os.path.isdir(file):
        file_name = path + file
        # img = cv2.imread(file_name)
        # plotImage(img, "test")
        # print(file_name)
        hog_vector = prepare_image(file_name)
        data_input.append(hog_vector)
        print("img done!")
    else:
        print("cannot open the file!")

# print("positive done!")
file_dir_2 = os.listdir(path_2)
for file in file_dir_2:
    if not os.path.isdir(file):
        file_name = path_2 + file
        hog_vector = prepare_image(file_name)
        data_input.append(hog_vector)
    else:
        print("cannot open the file!")
data_input = np.matrix(data_input)
print("done")

img done!
img done!
img done!
img done!
img done!
img done!
img done!
img done!
img done!
img done!
done


## **Read in the test data**

In [13]:
import os
test_input = [] 
path = "/Users/shenmengjie/Desktop/Computer Vision/project2/data/Test_Positive/"
path_2 = "/Users/shenmengjie/Desktop/Computer Vision/project2/data/Test_Negative/"

file_dir = os.listdir(path)
for file in file_dir:
    if not os.path.isdir(file):
        file_name = path + file
        # img = cv2.imread(file_name)
        # plotImage(img, "test")
        # print(file_name)
        hog_vector = prepare_image(file_name)
        test_input.append(hog_vector)
    else:
        print("cannot open the file!")

# print("positive done!")
file_dir_2 = os.listdir(path_2)
for file in file_dir_2:
    if not os.path.isdir(file):
        file_name = path_2 + file
        # print(file_name)
        # img = cv2.imread(file_name)
        # plotImage(img, "test")
        # print(file_name)
        hog_vector = prepare_image(file_name)
        test_input.append(hog_vector)
    else:
        print("cannot open the file!")
test_input = np.matrix(test_input)
print("done")

done


## **Train a neural network with layer size of 250 neurons**

In [24]:
from numpy import random
HOG_nn = nn(data_input, 250, test_input)
HOG_nn.train()

e-02 4.64701390e-27 5.47009709e-09
  3.36926875e-38 4.69615752e-34 2.83720352e-01 2.55776324e-01]]Loss[[0.22942169]]test loss:[[4.62003373]]
iteration3840:[[9.99996514e-01 1.00000000e+00 1.00000000e+00 9.99999990e-01
  9.99999990e-01 9.99409380e-01 9.98290872e-01 3.23901650e-02
  9.99999997e-01 9.99995268e-01 1.50928894e-13 1.73160495e-15
  2.94447827e-01 7.28142574e-02 4.76652465e-27 5.58483262e-09
  3.44802095e-38 4.80188811e-34 2.82418233e-01 2.55476621e-01]]Loss[[0.22417327]]test loss:[[4.61503073]]
iteration3850:[[9.99996629e-01 1.00000000e+00 1.00000000e+00 9.99999990e-01
  9.99999990e-01 9.99426454e-01 9.98344733e-01 3.58527129e-02
  9.99999997e-01 9.99995408e-01 1.54594465e-13 1.78358253e-15
  2.94078442e-01 7.30110105e-02 4.88790010e-27 5.70044656e-09
  3.52777036e-38 4.90879150e-34 2.81090971e-01 2.55133239e-01]]Loss[[0.21896047]]test loss:[[4.61009612]]
iteration3860:[[9.99996739e-01 1.00000000e+00 1.00000000e+00 9.99999990e-01
  9.99999990e-01 9.99442810e-01 9.98396232e-01 

In [25]:
result = HOG_nn.pred(test_input.T)
#print probability result
print(result)

[[7.96258897e-13 5.07085538e-06 1.09504089e-07 1.74108319e-01
  7.73218200e-13 4.33168518e-34 1.47117814e-01 3.44989732e-15
  9.84174832e-01 2.82480584e-09]]


In [26]:
result[result >= 0.5] = 1
result[result < 0.5] = 0
result

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

## **Train a neural network with layer size of 500 neurons** 

In [14]:
# random.seed(1)
HOG_nn_500 = nn(data_input, 500, test_input)
HOG_nn_500.train()

iteration0:[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]Loss0test loss:[[7.13903477]]
iteration10:[[9.98572569e-01 9.99999990e-01 1.00000000e+00 1.00000000e+00
  9.75612079e-01 1.52942996e-05 3.18569277e-10 9.99999990e-01
  1.00000000e+00 9.99999990e-01 1.09643916e-24 1.00000000e+00
  9.99999990e-01 1.00000000e+00 9.99999990e-01 1.46428766e-04
  1.02348810e-07 6.26479734e-18 9.99999990e-01 9.99999990e-01]]Loss[[8.38896581]]test loss:[[7.13244355]]
iteration20:[[9.98676733e-01 9.99999990e-01 1.00000000e+00 1.00000000e+00
  9.78426853e-01 2.03986939e-05 4.33199906e-10 9.99999990e-01
  1.00000000e+00 9.99999990e-01 1.14953688e-24 1.00000000e+00
  9.99999990e-01 1.00000000e+00 9.99999990e-01 1.55731458e-04
  1.10868322e-07 7.06440362e-18 9.99999990e-01 9.99999990e-01]]Loss[[8.3402005]]test loss:[[7.12594144]]
iteration30:[[9.98772230e-01 9.99999990e-01 1.00000000e+00 1.00000000e+00
  9.80894120e-01 2.71812326e-05 5.

In [18]:
result = HOG_nn_500.pred(test_input.T)
print("probability result:")
print(result)
result[result >= 0.5] = 1
result[result < 0.5] = 0
print(result)

probability result:
[[9.99999997e-01 3.12256353e-14 1.00000000e+00 1.50151626e-23
  1.00000000e+00 1.10860741e-06 1.00000000e+00 4.46865026e-36
  1.00000000e+00 1.00000000e+00]]
[[1. 0. 1. 0. 1. 0. 1. 0. 1. 1.]]


## **Train a neural network with layer size of 1000 neurons**

In [30]:
HOG_nn_1000 = nn(data_input, 1000, test_input)
HOG_nn_1000.train()

.99999990e-01 9.99999990e-01 9.99999990e-01 9.99999990e-01
  1.00000000e+00 4.70148491e-01 1.13187613e-21 9.99999990e-01]]Loss[[12.51871589]]test loss:[[9.17138265]]
iteration50:[[4.99229746e-48 9.99999990e-01 9.99999990e-01 9.99999990e-01
  9.99999990e-01 9.81480667e-01 9.99999990e-01 9.99999990e-01
  9.99999990e-01 9.99999990e-01 1.91821681e-10 9.99999990e-01
  9.99999990e-01 9.99999990e-01 9.99999990e-01 9.99999990e-01
  1.00000000e+00 3.59460034e-01 8.80499799e-22 9.99999990e-01]]Loss[[12.45580547]]test loss:[[9.14334148]]
iteration60:[[7.37377820e-48 9.99999990e-01 9.99999990e-01 9.99999990e-01
  9.99999990e-01 9.77360126e-01 9.99999990e-01 9.99999990e-01
  9.99999990e-01 9.99999990e-01 1.58397122e-10 9.99999990e-01
  9.99999990e-01 9.99999990e-01 9.99999990e-01 9.99999990e-01
  1.00000000e+00 2.82436438e-01 7.23017433e-22 9.99999990e-01]]Loss[[12.39685807]]test loss:[[9.11950893]]
iteration70:[[1.07833687e-47 9.99999990e-01 9.99999990e-01 9.99999990e-01
  9.99999990e-01 9.7205690

In [31]:
result = HOG_nn_1000.pred(test_input.T)
print("probability result:")
print(result)
result[result >= 0.5] = 1
result[result < 0.5] = 0
print(result)

probability result:
[[2.47667760e-09 9.99499761e-01 1.00000000e+00 7.38954563e-41
  9.89073402e-01 9.64289540e-01 6.64197595e-06 1.00000000e+00
  9.99997632e-01 1.00000000e+00]]
[[0. 1. 1. 0. 1. 1. 0. 1. 1. 1.]]
